In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import wbgapi as wb

# Clean the Given data

In [2]:
# Read the original data to python
data = pd.read_csv('homework_group_1-tidy.csv')

In [3]:
# If the country column is China.Hong.Kong,then change to Hong kong
## the match of hongkong is not correct, change the name to Hong Kong
data['country'] = np.where(data['country'] == 'China.Hong.Kong', 'Hong Kong',data['country'])

In [4]:
# Get the list of countries and use WBG matcher to match the country codes
countries = list(data['country'].unique())
country_code = wb.economy.coder(countries)
country_code = pd.DataFrame(country_code,index=[0])

In [5]:
# convert the country_code from row to column
country_code = country_code.transpose()
country_code = country_code.reset_index()
country_code = country_code.rename(columns={'index': 'country',0:'country code'})

In [6]:
# merge back to the original data
data = pd.merge(data,country_code,on='country')

In [7]:
# Seperate the year from the date
data['year'] = data['date'].apply(lambda x: x.split('-')[0])

# Get and clean the GDP data

In [8]:
# search for the gpd per capita data
wb.search("gdp per capita")

ID,Name,Field,Value
NY.GDP.PCAP.CD,GDP per capita (current US$),IndicatorName,GDP per capita (current US$)
NY.GDP.PCAP.CD,GDP per capita (current US$),Longdefinition,...GDP per capita is gross domestic product divided by midyear population. GDP is the sum of...
NY.GDP.PCAP.CN,GDP per capita (current LCU),IndicatorName,GDP per capita (current LCU)
NY.GDP.PCAP.CN,GDP per capita (current LCU),Longdefinition,...GDP per capita is gross domestic product divided by midyear population. GDP is the sum of...
NY.GDP.PCAP.KD,GDP per capita (constant 2015 US$),IndicatorName,GDP per capita (constant 2015 US$)
NY.GDP.PCAP.KD,GDP per capita (constant 2015 US$),Longdefinition,...GDP per capita is gross domestic product divided by midyear population. GDP is the sum of...
NY.GDP.PCAP.KD.ZG,GDP per capita growth (annual %),IndicatorName,GDP per capita growth (annual %)
NY.GDP.PCAP.KD.ZG,GDP per capita growth (annual %),Longdefinition,...Annual percentage growth rate of GDP per capita based on constant local currency. GDP per capita is gross domestic product...
NY.GDP.PCAP.KN,GDP per capita (constant LCU),IndicatorName,GDP per capita (constant LCU)
NY.GDP.PCAP.KN,GDP per capita (constant LCU),Longdefinition,...GDP per capita is gross domestic product divided by midyear population. GDP at purchaser's...


In [51]:
# Get gdp per capita data using WBG API
GDP = wb.data.DataFrame('NY.GDP.PCAP.PP.CD', list(data['country code'].unique()))
GDP['economy'] = GDP.index

In [52]:
# Change the dataframe format and rename
GDP = pd.melt(GDP, id_vars = 'economy',value_vars=list(GDP.columns))
GDP = GDP.rename(columns={'economy': 'country code', 'variable':'year','value':'GDP per Capita'})

In [53]:
# Rename the values in variable column, delete YR
GDP['year'] = GDP['year'].str[2:]

In [55]:
# merge the gdp data back to the original data
data2 = pd.merge(data,GDP,on=['country code','year'], how='left')